In [1]:
# !pip install -e /files/shared/ap/jupyter-notebook/pkgs/objFuncs --break-system-packages
# !pip install -e /files/shared/ap/jupyter-notebook/pkgs/pyBO --break-system-packages

In [2]:
# !pip install -e /files/shared/ap/jupyter-notebook/pkgs/objFuncs
# !pip install -e /files/shared/ap/jupyter-notebook/pkgs/pyBO

In [3]:
# !pip install -e ../../objFuncs
# !pip install -e ../../pyBO

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import time
import pandas as pd
from copy import deepcopy as copy

/home/devuser/.local/lib/python3.9/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [5]:
import objFuncs
from objFuncs import VM
from objFuncs import residuals
from objFuncs.util import plot_obj_history, plot_multi_obj_history
from objFuncs import flame_utils_kilean as flame_utils

objFuncs version: 1.0.3. updated on 2024-01-25


In [6]:
objFuncs._global_machineIO._fetch_data_time_span = 5.0
objFuncs._global_machineIO._ensure_set_timewait_after_ramp = 1.0
objFuncs._global_machineIO._test = True

In [ ]:
objFuncs._global_machineIO.fetch_data = machineIO_vms[0].fetch_data
objFuncs._global_machineIO.ensure_set = machineIO_vms[0].ensure_set


In [7]:
ensure_set = objFuncs._global_machineIO.ensure_set
fetch_data = objFuncs._global_machineIO.fetch_data

In [8]:
from pyBO import pyBO

# n_init = 16
# batch_size = 2
# n_local_search = batch_size
# n_loop = 8
# budget = n_init+n_loop*batch_size
# print(budget)

pyBO version: 1.0.0. updated on 2023-12-14


In [9]:
from_Dnum = 2369
to_Dnum = 2600

In [10]:
lat_file = "../objFuncs/FLAME_lat/test_FS1d_20231130_040841_read.lat"
fm = flame_utils.ModelFlame(lat_file)

In [20]:
def get_df_by_type(type_name,from_Dnum=from_Dnum, to_Dnum=to_Dnum, fm=fm):
    df = fm.get_df_by_type(type_name)
    names = [name for name in df['name']
             if from_Dnum <= flame_utils.get_Dnum_from_pv(name) <= to_Dnum]
    names = flame_utils.sort_by_Dnum(names)
    df = df[df['name'].isin(names)].sort_values(
            by='name', key=lambda x: x.map(names.index))
    PV = [ name.replace(
           'DCH','PSC2').replace(
           'DCV','PSC1').replace(
           'QH'  ,'PSQ' ).replace(
           'QV'  ,'PSQ' )
           for name in names]
    df['PV'] = PV
    return df

In [21]:
BPMdf = get_df_by_type('bpm')
BPMdf

,z,L,type,name,PV
index,,,,,
73,13.136087,0.0,bpm,FS1_CSS:BPM_D2369,FS1_CSS:BPM_D2369
82,14.587727,0.0,bpm,FS1_CSS:BPM_D2383,FS1_CSS:BPM_D2383
103,18.403514,0.0,bpm,FS1_BBS:BPM_D2421,FS1_BBS:BPM_D2421
123,22.884765,0.0,bpm,FS1_BBS:BPM_D2466,FS1_BBS:BPM_D2466
142,26.502032,0.0,bpm,FS1_BMS:BPM_D2502,FS1_BMS:BPM_D2502
167,29.926089,0.0,bpm,FS1_BMS:BPM_D2537,FS1_BMS:BPM_D2537
197,34.923571,0.0,bpm,FS1_BMS:BPM_D2587,FS1_BMS:BPM_D2587
203,36.294557,0.0,bpm,FS1_BMS:BPM_D2600,FS1_BMS:BPM_D2600


In [22]:
QUADdf = get_df_by_type('quadrupole')
CORdf  = get_df_by_type('orbtrim')

In [25]:
decisions = flame_utils.sort_by_Dnum( QUADdf.PV.tolist() + CORdf.PV.tolist() )
decision_CSETs = [pv+':I_CSET' for pv in decisions]
decision_RDs   = [pv+':I_RD'   for pv in decisions]
decision_CSET0,_ = fetch_data(decision_CSETs,0.1)

decision_min = [] 
decision_max = []
decision_tol = []
for v,k in zip(decision_CSET0,decisions):
    if 'PSC' in k:
        decision_min.append(max  (0,v -4))
        decision_min.append(min( 50,v +4))
        decision_tol.append(0.2)
    elif 'PSQ' in k:
        decision_min.append(max  (0,v-20))
        decision_min.append(min(165,v+20))
        decision_tol.append(0.4)
    else:
        raise ValueError('{k} is not COR or QUAD')

In [ ]:
beamcurrent_min = caget(beamcurrentPV)*0.95

In [27]:
monitor_PVs = []
for name in BPMdf.name:
    monitor_PVs += [name + ':XPOS_RD']
    monitor_PVs += [name + ':YPOS_RD']
    monitor_PVs += [name + ':MAG_RD']
    monitor_PVs += [name + ':CURRENT_RD']
    monitor_PVs += [name + ':PHASE_RD']
# monitor_PVs += [beamcurrentPV]
# monitor_PVs

In [28]:
def single_knob_scan(setPV,rdPV,tol,
                     x0,xmin,xmax,nscan,
                     beamcurrentPV=beamcurrentPV, 
                     beamcurrent_min=beamcurrent_min):
    assert beamcurrentPV in monitor_PVs
    xscan = np.linsapce(xmin,xmax,nscan)
    xscan1 = xscan[ xscan <  x0 ][::-1]
    xscan2 = xscan[ xscan >= x0 ]
    
    collect_mean = []
    collect_std  = []
    
    for x in xscan1:
        ensure_set([setPV],[rdPV],x,tol=tol)
        _,raw = fetch_data(decision_RDs + monitor_PVs,with_data=True)
        std = raw['std']
        if raw['mean'].loc['beamcurrentPV'] < beamcurrent_min:
            break
        collect_mean.append(raw['mean'])
        collect_std .append(raw['std' ])
    
    for x in xscan2:
        ensure_set([setPV],[rdPV],x,tol=tol)
        _,raw = fetch_data(decision_RDs + monitor_PVs,with_data=True)
        std = raw['std']
        if raw['mean'].loc['beamcurrentPV'] < beamcurrent_min:
            break
        collect_mean.append(raw['mean'])
        collect_std .append(raw['std' ])
        
    collect_mean = pd.DataFrame(collect_mean, columns= decision_RDs + monitor_PVs )
    collect_std  = pd.DataFrame(collect_std , columns= decision_RDs + monitor_PVs )
    
    ensure_set([setPV],[rdPV],x0,tol=tol)
    
    return collect_mean.to_dict(), collect_std.to_dict()

In [ ]:
collect_data = {}

for pv in (decisions[::-1], 
           decision_CSET0[::-1],
           decision_min[::-1],
           decision_max[::-1],
           decision_tol[::-1]
          ):
    collect_data['pv'] = {}
    collect_mean, collect_std = single_knob_scan(pv+":I_CSET",pv+":I_RD",tol,
                                                 x0,xmin,xmax,7,
                                                 )
    collect_data['pv']['mean'] = collect_mean
    collect_data['pv']['std' ] = collect_std
    
    pickle.dump(collect_data,open('FS1_calibration_data.pkl','wr'))